In [ ]:
# kutuphaneler
import requests
from parsel import Selector
import pandas as pd

# ilan ozelliklerinin kaydedildigi degisken
ilanlar = []

# linklerin kaydedildigi degisken
linkler = []

# 1 sayfadan baslayarak x sayfa kadar tarar, sitede en son 28 sayfa vardır
# yeni sonuc gelmeyince dongu durdurulur
for i in range(1, 20):
    resp = requests.get(f"https://www.hepsiemlak.com/duzce-satilik?page={i}")
    selectorLink = Selector(resp.text)
    # x sayfadaki linkler alinir
    tempLink = selectorLink.xpath('//div[@class="links"]/a/@href').getall()
    # yeni sonuc gelmediyse dongu kirilir
    if len(tempLink) < 1:
        break
    # sonuc varsa linklere eklenir
    linkler += tempLink

# ilanlar birer birer dongu ile gezilir
for link in linkler:
    try:
        resp = requests.get(f"https://www.hepsiemlak.com{link}")
        selector = Selector(resp.text)
        # fiyat bilgileri alinir ve gereksiz bosluklar strip ile temizlenir
        fiyat = selector.xpath('//p[@class="fontRB fz24 price"]/text()').get().strip()
        #  il ilce mahalle bilgileri alinir ve gereksiz bosluklar temizlenir
        kisaDetay = [i.replace(" ","").replace("\n","").replace("m","") for i in selector.xpath('//ul[@class="short-info-list"]/li/text()').getall()]
        il = kisaDetay[0]
        ilce = kisaDetay[1]
        mahalle = kisaDetay[2]
        # detayli ilan bilgileri alinir
        detaylar = [i.strip().replace("\n","") for i in selector.xpath('//ul[@class="adv-info-list"]/li/span/text()').getall()]
        # veriler alinirken net ve brut m2 ayri ayri gorunuyor bu veriler bulunur ve listede birlestirilir (cekilen veri ile ilgili)
        for i in detaylar:
            if i[0] == "/":
                birlestir = detaylar.index(i)
                break
        detaylar[birlestir-1] += detaylar[birlestir]
        detaylar.pop(birlestir)
        # detaylar sutun ve icerik bilgisi olarak ayrilir sirayla tek cift durumuna gore
        sutunlar = [detaylar[i] for i in range(len(detaylar)) if i%2==0]
        icerik = [detaylar[i] for i in range(len(detaylar)) if i%2==1]

        # ilan bilgileri tempilan degiskenine atilir ve eksik veri olmadigi durumda genel ilanlar listesine atilir
        tempilan = [fiyat, il, ilce, mahalle]
        # sutun yani ozelliklerde dongu ile gezinilir ve istenen bilgiler ozelliklere eklenir
        for i in range(len(sutunlar)):
            if sutunlar[i] == "İlan Durumu":
                tempilan.append(icerik[i])

            if sutunlar[i] == "Konut Şekli":
                tempilan.append(icerik[i])
                
            if sutunlar[i] == "Oda + Salon Sayısı":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Brüt / Net M2":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Bulunduğu Kat":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Bina Yaşı":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Banyo Sayısı":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Yapı Tipi":
                tempilan.append(icerik[i])

            if sutunlar[i] == "Yapının Durumu":
                tempilan.append(icerik[i])
            
            if sutunlar[i] == "Yakıt Tipi":
                tempilan.append(icerik[i])
        # ozelliklerin butunlugu kontrol edilir 14 ozelligin hepsi varsa genel listeye eklenir
        if len(tempilan) == 14:
            ilanlar.append(tempilan)
    except:
        pass

# dataframe olusturulur
sonuc = pd.DataFrame(data= ilanlar)
print(sonuc)
# ilgili dosyalara donusturulur cikti alinir
sonuc.to_csv("outputCSV.csv")
sonuc.to_excel("outputExcel.xlsx")
